In [26]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from konlpy.tag import Hannanum, Mecab, Kkma, Komoran, Okt
import time

In [67]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data2.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

,news,code
0,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,경제
1,▶제21대 총선 실시간 개표 현황 및 결과 보기\n\n총선에서 여당이 다시 한 번 ...,경제
2,[뉴욕=AP/뉴시스]지난 10일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다....,경제
3,"부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 1200원대, 경유는 1000원...",경제
4,담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다.이전...,경제


In [68]:
# 정규 표현식을 이용해 한글만 사용
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...
1       제대 총선 실시간 개표 현황 및 결과 보기총선에서 여당이 다시 한 번 승리를 거두면...
2       뉴욕뉴시스지난 일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다 신종 코로나바이...
3       부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 원대 경유는 원대에 돌입했다일 ...
4       담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다이전까...
                              ...                        
3698    레고켐바이오사이언스대표 김용주사진가 글로벌 기술수출에 또다시 성공했다 년 중국 포순...
3699    제대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 일 오후 시부...
3700    아마존 연합뉴스 자료사진 아마존 연합뉴스 자료사진직원들 코로나 확산 방지 조치 충분...
3701    제대 국회의원선거가 임박한 가운데 투표 및 개표 방송을 준비하는 기업들의 움직임이 ...
3702    롤 점검이 일 진행될 예정이다온라인게임 리그오브레전드이하 롤은 일 오전 시부터 오전...
Name: news, Length: 3703, dtype: object

In [69]:
# 데이터 Null값 확인
print(df.isnull().sum())

news    0
code    0
dtype: int64


In [70]:
# 중복 샘플 제거
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  2137


In [71]:
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    235
1     경제    902
2     사회    554
3  생활/문화    446


In [72]:
# 불용어 처리
stopwords = ['고','된다','이미','부터','에서','를','하다','한편','으로','각','총','사진','에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [73]:
# 토크나이즈 성능 비교를 위한 토크나이즈 설정
from konlpy.tag import Hannanum, Mecab, Kkma, Komoran, Okt

Mecab_t = Mecab()
Hannanum_t = Hannanum()
Kkma_t = Kkma()
Komoran_t = Komoran(userdic='/tmp/dic.txt')
Okt_t = Okt()

In [74]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수
def preprocessing(tokenizer, data):
  text_data = []

  for sentence in data:
    temp_data = []
    temp_data = tokenizer.morphs(sentence) 
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [75]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [76]:
start=time.time()

text_data = preprocessing(Mecab_t, df['news'])

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행
#- X_train은 TF-IDF 벡터, y_train은 레이블
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [77]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Mecab, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.00      0.00      0.00        59
          경제       0.63      0.98      0.77       233
          사회       0.92      0.93      0.93       132
       생활/문화       0.97      0.35      0.52       111

    accuracy                           0.73       535
   macro avg       0.63      0.57      0.55       535
weighted avg       0.70      0.73      0.67       535

[Mecab, preprocessing 걸린 시간] 2.9278676509857178 sec


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [78]:
start=time.time()

text_data = preprocessing(Hannanum_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [79]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Hannanum, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.00      0.00      0.00        59
          경제       0.61      0.98      0.75       233
          사회       0.92      0.92      0.92       132
       생활/문화       0.97      0.26      0.41       111

    accuracy                           0.71       535
   macro avg       0.62      0.54      0.52       535
weighted avg       0.69      0.71      0.64       535

[Hannanum, preprocessing 걸린 시간] 259.23154163360596 sec


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
start=time.time()
text_data = preprocessing(Kkma_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [19]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Kkma, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.90      0.75      0.82       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



In [21]:
start=time.time()

text_data = preprocessing(Komoran_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [22]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Komoran, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.90      0.75      0.82       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



In [24]:
start=time.time()

text_data = preprocessing(Okt_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [25]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Okt, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.91      0.71      0.80       236
          사회       0.78      0.93      0.85       422
       생활/문화       0.83      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.80      0.81       999
weighted avg       0.83      0.82      0.82       999

